In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w10p1/train.csv
/kaggle/input/2022-ai-w10p1/test.csv
/kaggle/input/2022-ai-w10p1/sample_submit.csv


In [2]:
import torch
import torch.optim as optim
import torchvision.transforms as transforms
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'


random.seed(42)
torch.manual_seed(42)
if device == 'cuda':
    torch.cuda.manual_seed_all(42)
    torch.cuda.manual_seed_all(42)
    print("cuda")
    
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

cuda


In [3]:
import torch
import torchvision.datasets as data
import torchvision.transforms as transforms
import random

In [4]:
train = pd.read_csv('/kaggle/input/2022-ai-w10p1/train.csv')
test = pd.read_csv('/kaggle/input/2022-ai-w10p1/test.csv')
sample = pd.read_csv('/kaggle/input/2022-ai-w10p1/sample_submit.csv')

In [5]:
# 학습 파라미터 설정
learning_rate = 0.001
training_epochs = 5
batch_size = 100
method = "alexnet"

In [6]:
train_y = np.asarray(train)[:, 0:1]
train_x = np.asarray(train.drop('Category', axis = 1))
test_x = np.asarray(test)

test_x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [7]:
import cv2
import torch
import torchvision
import torchvision.transforms as transforms

class Custom_Loader(torch.utils.data.Dataset):
    def __init__(self, data, label=None, split='train', transform=None):
        self.label = label
        self.split = split
        self.transform = transform
        
        # csv 파일로부터 얻어진 데이터를 28, 28, 1로 shape 변환
        data_1ch = np.reshape(data, (-1, 28, 28, 1)) # (60000, 28, 28, 1) 
        # 1ch 의 MNIST이미지를 3ch 로 변환 
        data_3ch = data_1ch.repeat(3, axis=-1) # (60000, 28, 28, 3) 
        # data 인스턴스에 변환한 데이터 저장
        self.data = data_3ch.astype(np.uint8)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # idx번째 데이터를 data에 저장
        data = self.data[idx]  # shape (28, 28, 3)
        
        # 28x28 이미지를 224x224로 resize
        data = cv2.resize(data, dsize=(224, 224), interpolation=cv2.INTER_LINEAR) # shape (224, 224, 3)
        
        # Tensor 변환 및 Normalization 
        data = self.transform(data)
        
        # train loader 일 경우 data, label 반환
        if self.split == 'train':
            return data, torch.from_numpy(self.label[idx])
        else:
        # test 일 경우, data만 반환 
            return data
        

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

train_dataset = Custom_Loader(train_x, train_y, split='train', transform=transform)
test_dataset = Custom_Loader(test_x, split='test', transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

In [8]:
import torchvision.models as models

if method == "resnet":
    model = models.resnet18(pretrained=True)
    model.fc = torch.nn.Linear(in_features=512, out_features=10, bias=True)
elif method == "vgg":
    model = models.vgg16(pretrained=True)
    model.classifier[6] = torch.nn.Linear(in_features=4096, out_features=10, bias=True)
elif method == "alexnet":
    model = models.alexnet(pretrained=True)
    model.classifier[6] = torch.nn.Linear(in_features=4096, out_features=10, bias=True)
else:
    raise ValueError("Check Method!")
    


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [9]:
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
loss = torch.nn.CrossEntropyLoss()

In [10]:
# 버전 2.0
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN,self).__init__() # 이부분 빼먹기 쉬운데 조심할 것

    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2,stride=2)
    )

    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2,stride=2)
    ) 

    self.fc = torch.nn.Linear(7*7*64,10,bias=True)
    torch.nn.init.kaiming_uniform(self.fc.weight)

  def forward(self,x):
    
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0),-1)     # out = Batch x 7 x 7 x 64
    out = self.fc(out)
    return out

In [11]:
total_batch = len(train_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in train_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = loss(hypothesis, Y.squeeze(-1))
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

[Epoch:    1] cost = 0.144369021
[Epoch:    2] cost = 0.0421045572
[Epoch:    3] cost = 0.0311104711
[Epoch:    4] cost = 0.0248560309
[Epoch:    5] cost = 0.0219418462
Learning Finished!


In [12]:
predict_arr = []
for X in test_loader:
        X = X.to(device)
        predict = model(X)
        predict_arr.append(predict.argmax(dim = 1))


In [13]:
l = []

for item in predict_arr:
    l2 = np.asarray(item.cpu().detach())
    for i in l2:
        l.append(i)


In [14]:
sample

,Id,Category
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
9995,9995,0
9996,9996,0
9997,9997,0
9998,9998,0


In [15]:
sample['Category'] = l
sample.to_csv('submit.csv', index = None)

In [16]:
l[101]

1

In [17]:
sample

,Id,Category
0,0,6
1,1,1
2,2,0
3,3,0
4,4,9
...,...,...
9995,9995,3
9996,9996,7
9997,9997,2
9998,9998,8
